In [4]:
import tensorflow as tf
from tensorflow.python.keras.datasets import cifar10
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import layers, activations, optimizers, models,losses
import os
import matplotlib.pyplot as plt

Model Parameters

In [2]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), "saved_models")
model_name = 'keras_cifar10_trained_model.h5'

# Load data into memory

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


Define Model

In [6]:
def model_fn(features, labels, mode):
    inputs = tf.convert_to_tensor(features["x"])

    #Convolution Layer-1 
    conv1 = tf.layers.conv2d(
        inputs=inputs,
        filters=32,
        kernel_size=[3, 3],
        activation=tf.nn.relu,
        padding="same",
        name="Conv-1.1"
    )
    conv1 = tf.layers.conv2d(
        inputs=conv1,
        filters=32,
        kernel_size=[3, 3],
        activation=tf.nn.relu,
        padding="same",
        name="Conv-1.2"
    )
    pool1 = tf.layers.max_pooling2d(conv1, pool_size=[2, 2], padding="same", strides=1,
                                    name="Max_Pool-1")
    dr1 = tf.layers.dropout(pool1, rate=0.25, name="drop-1")

    #Convolution Layer-2 
    conv2 = tf.layers.conv2d(
        inputs=dr1,
        filters=64,
        kernel_size=[3, 3],
        activation=tf.nn.relu,
        padding="same",
        name="Conv-2.1"
    )
    conv2 = tf.layers.conv2d(
        inputs=conv2,
        filters=64,
        kernel_size=[3, 3],
        activation=tf.nn.relu,
        padding="same",
        name="Conv-2.2"
    )
    pool2 = tf.layers.max_pooling2d(conv2, pool_size=[2, 2], padding="same", strides=1,
                                    name="Max_Pool-2")
    dr2 = tf.layers.dropout(pool2, rate=0.25, name="drop-2")

    pool2flat = tf.layers.flatten(dr2, name="flattern")
    dense = tf.layers.dense(inputs=pool2flat, units=512, activation=tf.nn.relu)
    dr_dense = tf.layers.dropout(dense, rate=0.25, name="drop-3")

    logits = tf.layers.dense(inputs=dr_dense, units=num_classes, activation=tf.nn.softmax,
                             name="Softmax Tensor")

    predictions = {
        "classes": tf.argmax(logits, axis=1),
        "probabilities": logits
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.softmax_cross_entropy(logits=logits, onehot_labels=labels)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.RMSPropOptimizer(learning_rate=0.0001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(mode=mode, eval_metric_ops=eval_metric, loss=loss)
